In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [64]:
freeFloats = pd.read_csv(r'extraCompanyDataFinal/freeFloatPercFinal.csv')
sharesOutstanding = pd.read_csv(r'extraCompanyDataFinal/outstandingSharesFinal.csv')

FTSE = pd.read_csv(r'FTSE17-20.csv')
FTSE['Date'] = pd.to_datetime(FTSE['Date'])

FTSEStocksEachQuarter = pd.read_csv(r'FTSEStocksEachQuarter.csv')

In [27]:
freeFloats['AAL'][0]
FTSEStocksEachQuarter.head()

,Unnamed: 0,"Q3, 17","Q4, 17","Q1, 18","Q2, 18","Q3, 18","Q4, 18","Q1, 19","Q2, 19","Q3, 19","Q4, 19"
0,0,III,III,III,III,III,III,III,III,III,III
1,1,ABF,ABF,ABF,ABF,ABF,ABF,ABF,ABF,ABF,ABF
2,2,ADM,ADM,ADM,ADM,ADM,ADM,ADM,ADM,ADM,ADM
3,3,AAL,AAL,AAL,AAL,AAL,AAL,AAL,AAL,AAL,AAL
4,4,ANTO,ANTO,ANTO,ANTO,ANTO,ANTO,ANTO,ANTO,ANTO,ANTO


In [32]:
#dates for share close prices between any two quarters - for index divisor
quarterCutOffDates = [datetime.strptime('Jun 19, 2017', '%b %d, %Y').date(), \
                         datetime.strptime('Sep 18, 2017', '%b %d, %Y').date(), \
                         datetime.strptime('Dec 18, 2017', '%b %d, %Y').date(), \
                         datetime.strptime('Mar 19, 2018', '%b %d, %Y').date(), \
                         datetime.strptime('Jun 18, 2018', '%b %d, %Y').date(), \
                         datetime.strptime('Sep 24, 2018', '%b %d, %Y').date(), \
                         datetime.strptime('Dec 24, 2018', '%b %d, %Y').date(), \
                         datetime.strptime('Mar 18, 2019', '%b %d, %Y').date(), \
                         datetime.strptime('Jun 24, 2019', '%b %d, %Y').date(), \
                         datetime.strptime('Sep 23, 2019', '%b %d, %Y').date()
                        ]

quarterNames = ['Q3, 17','Q4, 17', 'Q1, 18', 'Q2, 18', 'Q3, 18', 'Q4, 18', 'Q1, 19', 'Q2, 19', 'Q3, 19', 'Q4, 19']

quarterCutOffDates.reverse()
quarterNames.reverse()

len(quarterNames)
quarterNames[0][0:2]
#quarterCutOffDates

'Q4'

In [39]:
indexDivisors = pd.DataFrame({'Quarter CO Date': quarterCutOffDates}, index = quarterNames)
indexDivisors

,Quarter CO Date
"Q4, 19",2019-09-23
"Q3, 19",2019-06-24
"Q2, 19",2019-03-18
"Q1, 19",2018-12-24
"Q4, 18",2018-09-24
"Q3, 18",2018-06-18
"Q2, 18",2018-03-19
"Q1, 18",2017-12-18
"Q4, 17",2017-09-18
"Q3, 17",2017-06-19


In [65]:
indexDivisorsList = []
for idx, quarter in enumerate(indexDivisors.index.values):
    sumOfMarketCaps = 0
    for stock in FTSEStocksEachQuarter[quarter].values:
        if stock == 'AV.':
            stock = 'AV.L'
            stockPrice = pd.read_csv(f'stockCSVs/{stock}17-20.csv')
        elif stock[-1] == '.':
            stockPrice = pd.read_csv(f'stockCSVs/{stock}L17-20.csv')
        else:
            stockPrice = pd.read_csv(f'stockCSVs/{stock}.L17-20.csv')
        stockPrice['Date'] = pd.to_datetime(stockPrice['Date'])
        #print(stock)
        #print(freeFloats[stock][idx])
        #print(sharesOutstanding[stock][idx])
        #print(stockPrice[stockPrice['Date'] == indexDivisors['Quarter CO Date'][quarter]])
        #print(stockPrice[stockPrice['Date'] == indexDivisors['Quarter CO Date'][quarter]]['Close'].values[0])
        try:
            marketCap = freeFloats[stock][idx]*sharesOutstanding[stock][idx]*stockPrice[stockPrice['Date'] == indexDivisors['Quarter CO Date'][quarter]]['Close'].values[0]
        except IndexError:
            continue
        sumOfMarketCaps +=marketCap
    
    #print(FTSE[FTSE['Date'] == indexDivisors['Quarter CO Date'][quarter]])
    #print(FTSE[FTSE['Date'] == indexDivisors['Quarter CO Date'][quarter]]['Close'].values[0])
    indexDivisorsList.append(sumOfMarketCaps/(FTSE[FTSE['Date'] == indexDivisors['Quarter CO Date'][quarter]]['Close'].values[0])) 
    
indexDivisors['Divisors'] = indexDivisorsList    
#indexDivisors.to_csv('indexDivisors.csv')

,Quarter CO Date,Divisors
"Q4, 19",2019-09-23,2.752744e+10
"Q3, 19",2019-06-24,2.783326e+10
"Q2, 19",2019-03-18,2.962895e+10
"Q1, 19",2018-12-24,2.988978e+10
"Q4, 18",2018-09-24,2.786064e+10
"Q3, 18",2018-06-18,2.784479e+10
"Q2, 18",2018-03-19,2.740565e+10
"Q1, 18",2017-12-18,2.751398e+10
"Q4, 17",2017-09-18,2.713623e+10
"Q3, 17",2017-06-19,2.659119e+10


In [ ]:
indexDivisors.to_csv('indexDivisors.csv')

In [63]:
print(stock)

JET


In [42]:
stockPrice = pd.read_csv(f'stockCSVs/AAL.L17-20.csv')
stockPrice['Date'] = pd.to_datetime(stockPrice['Date'])
stockPrice['Date'][0] > indexDivisors['Quarter CO Date'][0]
stockPrice[stockPrice['Date'] == datetime.strptime('Jun 18, 2018', '%b %d, %Y').date()]['Close'].values[0]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
  after removing the cwd from sys.path.


1733.0